In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.autograd import Variable


In [36]:
#device setup
device = 'cpu'
cuda = True if torch.cuda.is_available() else False
FloatTensor = torch.FloatTensor
LongTensor = torch.LongTensor

In [37]:
#set up seed for consistancy
torch.manual_seed(1)
#hyperparameters based on slides
learning_rate = 2e-4
batch_size = 64
image_size = 64
#black and white 1 for color img 3(r.g.b)
image_channels = 3
noise_dimension = 100
epochs = 1
discriminator_features = 64
generator_features = 64
adam_beta = 0.5
latent_dim = 100
n_classes = 10
#get Cifar10 dataset
workers = 2
transforms = transforms.Compose(
    [
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transforms.Normalize(
            [0.5 for _ in range(image_channels)], [0.5 for _ in range(image_channels)]
        ),
    ]
)
data_train = torchvision.datasets.CIFAR10(root='./DL HW 3/data', 
                                           train=True, 
                                           transform=transforms,  
                                           download=True)

train_loader = torch.utils.data.DataLoader(data_train, batch_size=batch_size,
                                          shuffle=True,num_workers=workers)


Files already downloaded and verified


In [38]:
class Discriminator(nn.Module):
    def __init__(self, channels_img, features_d):
        super(Discriminator, self).__init__()
        self.discrim = nn.Sequential(
            # input: N x channels_img x 64 x 64
            nn.Conv2d(3, features_d, 4, 2, 1), 
            nn.LeakyReLU(0.2, inplace=True), 
            nn.Dropout2d(0.25),
            # _block(in_channels, out_channels, kernel_size, stride, padding)
            self._descrimStructure(features_d, features_d * 2, 4, 2, 1),
            self._descrimStructure(features_d * 2, features_d * 4, 4, 2, 1),
            self._descrimStructure(features_d * 4, features_d * 8, 4, 2, 1),
            # After all _block img output is 4x4 (Conv2d below makes into 1x1)
            
        )
       
        self.adv_layer = nn.Sequential(
                                nn.Conv2d(features_d * 8, 1, kernel_size=4, stride=1, padding=0),
                                nn.Sigmoid())
        self.aux_layer = nn.Sequential(
                                nn.Conv2d(features_d * 8, 10, 4, 1, 0, bias = False), 
                                nn.LogSoftmax(dim = 1))

    def _descrimStructure(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.Conv2d(
                in_channels,
                out_channels,
                kernel_size,
                stride,
                padding,
                bias=False,
            ),
            #nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout2d(0.25),
        )

    def forward(self, x):
        x = self.discrim(x)
        adv = self.adv_layer(x).view(-1)
        aux = self.aux_layer(x).view(-1,10)
        return adv,aux


class Generator(nn.Module):
    def __init__(self, channels_noise, channels_img, features_g):
        super(Generator, self).__init__()
        self.init_size = image_size   # Initial size before upsampling
        #self.l1 = nn.Sequential(nn.Linear(latent_dim, 128 * self.init_size ** 2))
        self.genet = nn.Sequential(
            nn.ConvTranspose2d(latent_dim+100,features_g* 16, 4, 1, 0, bias = False),
            # Input: N x channels_noise x 1 x 1
            self._genetStructure(features_g* 16, features_g* 8, 4, 2, 1),  
            self._genetStructure(features_g* 8, features_g* 4, 4, 2, 1), 
            self._genetStructure(features_g* 4, features_g* 2, 4, 2, 1),
            nn.ConvTranspose2d(features_g* 2, 3, 4, 2, 1),
            # Output: N x channels_img x 64 x 64
            nn.Tanh(),
        )
        self.embedding_layer = nn.Embedding(10, 100)

    def _genetStructure(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.ConvTranspose2d(
                in_channels,
                out_channels,
                kernel_size,
                stride,
                padding,
                bias=False,
            ),
            nn.BatchNorm2d(out_channels,0.8),
            nn.LeakyReLU(0.2, inplace=True),
        )

    def forward(self, noise, labels):
        embed = self.embedding_layer(labels).unsqueeze(2).unsqueeze(3)
        x = torch.cat([noise, embed], dim=1)
        return self.genet(x)

In [39]:
def model_weights(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

In [41]:
import gc

genet = Generator(noise_dimension, image_channels, generator_features).to(device)
discrim = Discriminator(image_channels, discriminator_features).to(device)
#genet.summary()
#discrim.summary()
print(genet)
model_weights(genet)
model_weights(discrim)
gen_loss = []
disc_loss = []
opt_gen = optim.Adam(genet.parameters(), lr=learning_rate, betas=(0.5,0.999))
opt_disc = optim.Adam(discrim.parameters(), lr=learning_rate, betas=(0.5,0.999))
criterion = nn.BCELoss()

fixed_noise = torch.randn(10, noise_dimension, 1, 1).to(device)
sample_label = torch.randint(0,10,(10,),dtype = torch.long).to(device)
step = 0

genet.train()
discrim.train()
fakeimg_list=[]
img_list=[]
for epoch in range(epochs):
    # Target labels not needed! <3 unsupervised
    for batch_idx, (real, _) in enumerate(train_loader):
        gc.collect()
        torch.cuda.empty_cache()
        real = real.to(device)
        noise = torch.randn(batch_size, noise_dimension, 1, 1).to(device)
        gen_labels = Variable(LongTensor(np.random.randint(0, n_classes, 64))).to(device)
      
        print(noise.shape)
        fake = genet(noise,gen_labels)

        ### Train Discriminator: max log(D(x)) + log(1 - D(G(z)))
        disc_real,disc_label = discrim(real)
        loss_disc_real = criterion(disc_real.reshape(-1), torch.ones_like(disc_real))
        disc_fake,disc_label_fk = discrim(fake.detach())
        loss_disc_fake = criterion(disc_fake.reshape(-1), torch.zeros_like(disc_fake))
        loss_disc = (loss_disc_real + loss_disc_fake) / 2
        discrim.zero_grad()
        loss_disc.backward()
        opt_disc.step()

        ### Train Generator: min log(1 - D(G(z))) <-> max log(D(G(z))
        output,lbel = discrim(fake)
        loss_gen = criterion(output.reshape(-1), torch.ones_like(output))
        genet.zero_grad()
        loss_gen.backward()
        opt_gen.step()
        gen_loss.append(loss_gen)
        disc_loss.append(loss_disc)
        
        # Print losses occasionally and print to tensorboard
        if 0 % 10 == 0:
            print(
                f"Epoch [{epoch}/{epochs}] Batch {batch_idx}/{len(train_loader)} \
                  Loss D: {loss_disc:.4f}, loss G: {loss_gen:.4f}"
            )

            with torch.no_grad():
                fak = genet(fixed_noise,sample_label).detach().cpu()
                realI = real.detach().cpu()
                fakeimg_list.append(torchvision.utils.make_grid(fak, padding=2, normalize=True))
                img_list.append(torchvision.utils.make_grid(realI, padding=2, normalize=True))

            step += 1

Generator(
  (genet): Sequential(
    (0): ConvTranspose2d(200, 1024, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): Sequential(
      (0): Upsample(scale_factor=2.0, mode=nearest)
      (1): ConvTranspose2d(1024, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (2): BatchNorm2d(512, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
      (3): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (2): Sequential(
      (0): Upsample(scale_factor=2.0, mode=nearest)
      (1): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (2): BatchNorm2d(256, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
      (3): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (3): Sequential(
      (0): Upsample(scale_factor=2.0, mode=nearest)
      (1): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (2): BatchNorm2d(128, eps=0.8, momentum=0.1, affine=Tru

RuntimeError: [enforce fail at alloc_cpu.cpp:73] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 268435456 bytes. Error code 12 (Cannot allocate memory)